In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 

In [2]:
!pip install openai

In [3]:
# Base ctransformers with no GPU acceleration
!pip install ctransformers>=0.2.24
# Or with CUDA GPU acceleration
!pip install ctransformers[cuda]>=0.2.24
# Or with ROCm GPU acceleration
!CT_HIPBLAS=1 pip install ctransformers>=0.2.24 --no-binary ctransformers
# Or with Metal GPU acceleration for macOS systems
!CT_METAL=1 pip install ctransformers>=0.2.24 --no-binary ctransformers


In [40]:
import os
import torch
import json
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import openai
import pandas as pd
from datasets import Dataset

In [30]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# Fine-tuned model name
new_model = "llama-2-7b-qlora"

In [42]:
#dataset loading

# Specify the path to your JSONL file
jsonl_file_path = "data/finetuning_data.jsonl"

# Read the JSONL file into a DataFrame
df = pd.read_json(jsonl_file_path, lines=True)
dataset = Dataset.from_pandas(df)


In [35]:
dataset.head()

,dialogues,summary
0,Monica Geller : There's nothing to tell! He's ...,"Monica is going out with someone, but her frie..."
1,Joey Tribbiani : Instead of...? Chandler Bing ...,"Joey and Chandler discuss having dreams, while..."
2,Ross Geller : I just feel like someone reached...,Ross has an unsettling sensation like someone ...
3,Monica Geller : No you don't. Ross Geller : No...,Ross Geller's ex-wife left him and he is strug...
4,"Ross Geller : I don't want to be single, okay?...",Rachel discovers she is more attracted to a gr...


In [26]:

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [17]:
# Load dataset (you can process it here)
file_path = 'data/finetuning_data.jsonl'

with open(file_path) as f:
    dataset = pd.DataFrame(json.loads(line) for line in f)

In [18]:
dataset.head()

,dialogues,summary
0,Monica Geller : There's nothing to tell! He's ...,"Monica is going out with someone, but her frie..."
1,Joey Tribbiani : Instead of...? Chandler Bing ...,"Joey and Chandler discuss having dreams, while..."
2,Ross Geller : I just feel like someone reached...,Ross has an unsettling sensation like someone ...
3,Monica Geller : No you don't. Ross Geller : No...,Ross Geller's ex-wife left him and he is strug...
4,"Ross Geller : I don't want to be single, okay?...",Rachel discovers she is more attracted to a gr...


In [24]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


In [29]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

In [36]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [37]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [44]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type
)

In [45]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="dialogues",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

Map:   0%|          | 0/6134 [00:00<?, ? examples/s]

In [46]:
# Train model
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /common/home/za224/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/common/home/za224/.cache/pypoetry/virtualenvs/persona_llm_for_qa_chatbot-s35W1huD-py3.10/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/common/home/za224/.cache/pypoetry/virtualenvs/persona_llm_for_qa_chatbot-s35W1huD-py3.10/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scala

Step,Training Loss
25,2.263600
50,1.967800
75,2.027700
100,1.888500
125,1.995800
150,1.857900
175,1.978200
200,1.852500
225,1.974200
250,1.843500


/common/home/za224/.cache/pypoetry/virtualenvs/persona_llm_for_qa_chatbot-s35W1huD-py3.10/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/common/home/za224/.cache/pypoetry/virtualenvs/persona_llm_for_qa_chatbot-s35W1huD-py3.10/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/common/home/za224/.cache/pypoetry/virtualenvs/persona_llm_for_qa_chatbot-s35W1huD-py3.10/lib/python3.10/site-pac

TrainOutput(global_step=384, training_loss=1.9399057775735855, metrics={'train_runtime': 2830.0815, 'train_samples_per_second': 2.167, 'train_steps_per_second': 0.136, 'total_flos': 2.692896215875584e+16, 'train_loss': 1.9399057775735855, 'epoch': 1.0})

In [47]:
# Save trained model
trainer.model.save_pretrained(new_model)